In [16]:
import numpy as np
import pandas as pd
import datetime as dt
import itertools
from more_itertools import set_partitions

In [17]:
pd.options.display.max_rows = 12
truck_speed = 60
working_hours = 12
m = 7 # Number of trucks
#get a matrix of distances between locations
df = pd.read_csv('dist_vologda_matrix.csv', sep=',')
df = df.iloc[:, 1:]
df = df / 1000
df

,АЗС №35400 Кичменгский Городок,АЗС №35401 Великий Устюг-1,АЗС №35402 Великий Устюг-2,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35406 Журавлево,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,99.2272,101.8513,139.2135,154.2522,777.2635,102.4328,520.5020,517.8863,266.7014,...,610.3886,610.3028,607.3283,609.0579,611.0852,611.0605,604.2684,605.7555,502.9826,477.9325
1,99.0684,0.0000,4.9465,134.5201,149.5589,748.0318,5.5280,491.2703,488.6546,241.7276,...,581.1570,581.0711,578.0967,579.8263,581.8535,581.8288,575.0368,576.5238,473.7510,448.7008
2,101.8148,5.0688,0.0000,137.2665,152.3053,750.7781,3.0302,494.0166,491.4010,244.4739,...,583.9033,583.8174,580.8430,582.5726,584.5999,584.5752,577.7831,579.2702,476.4973,451.4472
3,139.2135,134.6791,137.3031,0.0000,15.0961,613.5690,137.8846,356.8075,354.1918,107.2648,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
4,154.2522,149.7178,152.3419,15.0961,0.0000,613.8947,152.9234,357.1332,354.5176,107.5905,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,611.7861,582.6174,585.2414,447.9957,448.3214,191.2351,585.8229,181.8523,179.2367,345.2112,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
60,605.7962,576.6275,579.2515,442.0057,442.3315,185.0682,579.8330,175.8624,173.2467,339.2213,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
61,607.1467,577.9779,580.6020,443.3562,443.6820,186.4186,581.1835,177.2129,174.5972,340.5717,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
62,504.6641,475.4954,478.1194,340.8736,341.1994,274.3706,478.7009,74.7303,72.1146,238.0892,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [18]:
#remove locations, which cannot be served in 12 hours (driver can't get there and come back in 12 hours)

to_remove = ((2 * (df.values[-1,:] / truck_speed) < working_hours) == False).nonzero()[0]
df = df.drop(df.index[to_remove])
df = df.drop(df.columns[to_remove], axis=1)
df = df.reset_index(drop=True)
df

,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,АЗС №35410 Сазоново-1,АЗС №35411 Сазоново-2,АЗС №35412 Вытегра,АЗС №35413 Липник,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,15.0961,613.5690,356.8075,354.1918,107.2648,619.0077,618.5201,629.3182,472.6696,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
1,15.0961,0.0000,613.8947,357.1332,354.5176,107.5905,619.3335,618.8458,629.6440,472.9954,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
2,615.1980,615.5238,0.0000,349.0547,346.4390,512.4135,142.9597,142.4720,165.8965,193.2966,...,187.4480,187.3622,184.7850,188.3816,183.6881,191.1772,185.1426,186.6296,274.4141,309.3918
3,356.1259,356.4516,349.7479,0.0000,3.7018,253.3414,355.1867,354.6990,373.5276,216.8790,...,182.8731,182.7873,179.8128,181.5424,183.5697,183.5450,176.7529,178.2400,75.4671,45.6714
4,353.5507,353.8764,347.1727,3.7422,0.0000,250.7662,352.6115,352.1238,370.9524,214.3038,...,180.2979,180.2120,177.2376,178.9672,180.9945,180.9698,174.1777,175.6648,72.8919,43.0962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,447.9957,448.3214,191.2351,181.8523,179.2367,345.2112,196.6738,196.1861,332.9386,176.2900,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
52,442.0057,442.3315,185.0682,175.8624,173.2467,339.2213,190.5069,190.0192,326.9487,170.3001,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
53,443.3562,443.6820,186.4186,177.2129,174.5972,340.5717,191.8574,191.3697,328.2991,171.6506,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
54,340.8736,341.1994,274.3706,74.7303,72.1146,238.0892,279.8093,279.3216,346.5072,189.8586,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [19]:
from numpy.random import default_rng

#shrink array
rng = default_rng()
idx = rng.choice(len(df.values[0])-1, size=len(df.values[0]) - 1 - 10, replace=False)
print(idx)
df = df.drop(df.index[idx])
df = df.drop(df.columns[idx], axis=1)
df = df.reset_index(drop=True)
N = len(df.values[0])
df

[ 4 23 42  2 51 52 13 54 32 10 18 31 11  7 36 17 15 43 12  6 49 46 34  9
 39 45 53  8 30 24 25 19  5 14 22 50 35 40 29 44 28  3  0 47 38]


,АЗС №35404 Нюксеница,АЗС №35420 Шексна,АЗС №35427 Марьинское,АЗС №35429 Чекшино-1,АЗС №35434 Верховажье,АЗС №35435 Устюжна,АЗС №35441 Преображенского,АЗС №35445 Чернышевского-1,АЗС №35449 Нелазское,АЗС №35456 Маяковская,Нефтебаза
0,0.0000,391.4372,308.8242,244.9984,158.4343,551.6265,315.2839,305.8532,448.4384,443.9596,314.5638
1,392.4223,0.0000,82.8181,147.5906,304.4975,162.4712,77.6788,86.4086,59.2831,54.8043,86.2904
2,309.7320,82.6843,0.0000,64.9002,221.8072,242.8735,10.9076,8.0424,139.6855,135.2067,15.8192
3,244.8612,146.6791,64.0661,0.0000,156.9364,306.8684,70.5258,61.0950,203.6803,199.2015,69.8057
4,158.2224,303.4982,220.8852,157.0594,0.0000,463.6875,227.3449,217.9142,360.4994,356.0206,226.6248
5,553.2976,160.8752,243.6933,308.4658,465.3728,0.0000,238.5540,247.2839,103.5422,122.5588,247.1656
6,314.9784,78.0459,9.1574,70.1467,227.0536,238.2352,0.0000,9.1884,135.0471,130.5683,9.0701
7,306.0214,87.3570,8.1652,61.1897,218.0966,247.5463,9.4640,0.0000,144.3582,139.8794,8.7439
8,449.9896,57.5673,140.3854,205.1579,362.0648,103.6256,135.2461,143.9759,0.0000,18.9999,143.8577
9,445.0346,52.6123,135.4304,200.2029,357.1098,122.5360,130.2910,139.0209,19.3480,0.0000,138.9026


In [30]:
list(idx)

[4,
 23,
 42,
 2,
 51,
 52,
 13,
 54,
 32,
 10,
 18,
 31,
 11,
 7,
 36,
 17,
 15,
 43,
 12,
 6,
 49,
 46,
 34,
 9,
 39,
 45,
 53,
 8,
 30,
 24,
 25,
 19,
 5,
 14,
 22,
 50,
 35,
 40,
 29,
 44,
 28,
 3,
 0,
 47,
 38]

In [20]:
dest = np.arange(N - 1)
dest

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [21]:
import math
math.factorial(N-1)

3628800

In [22]:
def GetLength(route):
    if len(route) < 2:
        return df.values[-1, route[0]] + df.values[route[0], -1]
    length = df.values[-1, route[0]]
    for i in range(1, len(route)):
        length += df.values[route[i-1], route[i]]
    length += df.values[route[-1], -1]
    
    return length

In [23]:
GetLength((0,))

629.0949

In [24]:
from itertools import combinations
combs = [np.split(dest, idx) for n_splits in range(m) for idx in combinations(range(1, len(dest)), n_splits)]
print(combs[7])
print("\n")
print(combs[-1])

[array([0, 1, 2, 3, 4, 5, 6]), array([7, 8, 9])]


[array([0, 1, 2, 3]), array([4]), array([5]), array([6]), array([7]), array([8]), array([9])]


In [25]:
to_del = []
for i in range(len(combs)):
    for j in range(len(combs[i])):
        length = GetLength(combs[i][j])
        if (length / truck_speed) > working_hours:
            to_del.append(i)
len(to_del)

510

In [26]:
new_combs = []
for i in range(len(combs)):
    if i not in to_del:
        new_combs.append(combs[i])
len(new_combs)

83

In [27]:
max = 0
for comb in new_combs:
    if len(comb) > max:
        max = len(comb)
max

7

In [28]:
short_dist = working_hours * truck_speed * m
shortest = 0
for i in range(len(combs)):
    dist = 0
    route = []
    for j in range(len(combs[i])):
        perms = list(itertools.permutations(combs[i][j]))
        short_perm = perms[0]
        for perm in perms:
            if GetLength(perm) < GetLength(short_perm):
                short_perm = perm
        dist += GetLength(short_perm)
        route.append(short_perm)
    if dist < short_dist:
        short_dist = dist
        shortest = i
        short_route = route
print("Shortest solution: ", short_route)
print("It's total distance is ", short_dist)

Shortest solution:  [(7, 3, 4, 0, 2, 5, 8, 9, 1, 6)]
It's total distance is  1198.6906


In [29]:
dist = 0
for i in range(len(short_route)):
    dist += GetLength(short_route[i])
    print("Truck #{:} | Time: {:} | Distance: {:}".format(i, GetLength(new_combs[shortest][i]) / truck_speed, GetLength(new_combs[shortest][i])))
dist   

Truck #0 | Time: 10.484915 | Distance: 629.0949


1198.6906